##### Import Libraries

- To run this script we need the libraries below
- Pandas : For creating DataFrames
- Numpy:  For computations 
- OS : For modifying working directories 
- censusdata: This is a library for accessing US Census Bureau, we use some of its function to preview variables *
- Census: Main library for accessing the US Census Bureau API *
- pd.set_option allows us to view our entire output instead of it being truncated in jupyter notebook



 * You need to install these packages before you can use. Census = pip install census | censusdata = pip install CensusData

In [1]:
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
import warnings

pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

warnings.filterwarnings("ignore", message=".*DataFrame is highly fragmented.*")

##### Set Directory 

- Jupyter notebooks have default directories 
- Use os.chdir to set your preferred directory

In [2]:
pwd

'C:\\Users\\Thepr\\Downloads'

##### Pass your API Key

- Go to this [website](https://api.census.gov/data/key_signup.html 'sign up for your personal API key') and sign up your personal API key 

- Once you get your API key, call the Census function and pass your API key. Eg. Census("API Key") 

In [3]:
c = Census("e0577a26a616f4dda60446eae987e3b6d0d944a3")

#### Preview ACS variable 
- Use the censustable function from censusdata library to preview the census variables
- To preview specify the dataset type eg. ACS year 5 estimates (acs5), then the year of interest(2019), then the table number('B21001')
- Use the printtable function to format the output nicely 

In [4]:
cs.printtable(cs.censustable('acs5',2019,'DP02'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
DP02_0001E   | SELECTED SOCIAL CHARACTERISTIC | !! !! Estimate HOUSEHOLDS BY TYPE Total households       | int  
DP02_0001PE  | SELECTED SOCIAL CHARACTERISTIC | !! !! Percent HOUSEHOLDS BY TYPE Total households        | int  
DP02_0002E   | SELECTED SOCIAL CHARACTERISTIC | !! !! !! Estimate HOUSEHOLDS BY TYPE Total households Ma | int  
DP02_0002PE  | SELECTED SOCIAL CHARACTERISTIC | !! !! !! Percent HOUSEHOLDS BY TYPE Total households Mar | float
DP02_0003E   | SELECTED SOCIAL CHARACTERISTIC | !! !! !! !! Estimate HOUSEHOLDS BY TYPE Total households | int  
DP02_0003PE  | SELECTED SOCIAL CHARACTERISTIC | !! !! !! !! Percent HOUSEHOLDS BY TYPE Total households  | float
DP02_0004E   | SELECTED SOCIAL CHARACTERISTIC | !! !! !! Estimate HOUSEHOLDS BY TYPE Total ho

## Get Data 

- **ACS 5-Year Estimate**

In [5]:
Years = [2012, 2022]
AllData = []

for year in Years:
    ACS = c.acs5.get(['B05002_001E', 'B05002_002E', 'B05002_013E'],{'for': 'place:04000', 'in': 'state:48'},year=year)
    
    Data = pd.DataFrame(ACS)
    Data['year'] = year
    
    Data.rename(columns={'B05002_001E': 'Total Population','B05002_002E': 'US Native','B05002_013E': 'Foreign Born'}, 
                inplace=True)
    
    Data = Data[['year', 'Total Population', 'US Native', 'Foreign Born']]
    Data = Data.astype({'Total Population': int, 'US Native': int, 'Foreign Born': int})
    
    Data['% US Native'] = Data['US Native'] / Data['Total Population'] * 100
    Data['% Foreign Born'] = Data['Foreign Born'] / Data['Total Population'] * 100
    
    AllData.append(Data)

Datacombined = pd.concat(AllData).sort_values(by='year').reset_index(drop=True)

row2012 = Datacombined.loc[Datacombined['year'] == 2012].iloc[0]
row2022 = Datacombined.loc[Datacombined['year'] == 2022].iloc[0]

Datasummary = pd.DataFrame([
    {
        'Description': 'Total Population',
        '2012': row2012['Total Population'],
        '2022': row2022['Total Population'],
        'Absolute Change (2012-2022)': row2022['Total Population'] - row2012['Total Population'],
        'Percent Change (2012-2022)': ((row2022['Total Population'] - row2012['Total Population']) / 
                                       row2012['Total Population'] * 100)
    },
    {
        'Description': 'US Native',
        '2012': row2012['US Native'],
        '2022': row2022['US Native'],
        'Absolute Change (2012-2022)': row2022['US Native'] - row2012['US Native'],
        'Percent Change (2012-2022)': ((row2022['US Native'] - row2012['US Native']) / row2012['US Native'] * 100)
    },
    {
        'Description': 'Foreign Born',
        '2012': row2012['Foreign Born'],
        '2022': row2022['Foreign Born'],
        'Absolute Change (2012-2022)': row2022['Foreign Born'] - row2012['Foreign Born'],
        'Percent Change (2012-2022)': ((row2022['Foreign Born'] - row2012['Foreign Born']) /
                                       row2012['Foreign Born'] * 100)
    },
    {
        'Description': '% US Native',
        '2012': row2012['% US Native'],
        '2022': row2022['% US Native'],
        'Absolute Change (2012-2022)': row2022['% US Native'] - row2012['% US Native'],
        'Percent Change (2012-2022)': ((row2022['% US Native'] - row2012['% US Native']) /
                                       row2012['% US Native'] * 100)
    },
    {
        'Description': '% Foreign Born',
        '2012': row2012['% Foreign Born'],
        '2022': row2022['% Foreign Born'],
        'Absolute Change (2012-2022)': row2022['% Foreign Born'] - row2012['% Foreign Born'],
        'Percent Change (2012-2022)': ((row2022['% Foreign Born'] - row2012['% Foreign Born']) / 
                                       row2012['% Foreign Born'] * 100)
    }
])

# Round all numeric columns to zero decimals and convert to int
for col in ['2012', '2022', 'Absolute Change (2012-2022)', 'Percent Change (2012-2022)']:
    Datasummary[col] = Datasummary[col].round(0).astype(int)

Datasummary.to_csv('Foreign Born 5-Year Estimate 2012-2022.csv', index = False)


- **ACS 2013 to 2023**

In [6]:
Years = [2013, 2023]
AllData = []

for year in Years:
    ACS = c.acs5.get(['B05002_001E', 'B05002_002E', 'B05002_013E'],{'for': 'place:04000', 'in': 'state:48'},year=year)
    
    Data = pd.DataFrame(ACS)
    Data['year'] = year
    
    Data.rename(columns={'B05002_001E': 'Total Population','B05002_002E': 'US Native','B05002_013E': 'Foreign Born'}, 
                inplace=True)
    
    Data = Data[['year', 'Total Population', 'US Native', 'Foreign Born']]
    Data = Data.astype({'Total Population': int, 'US Native': int, 'Foreign Born': int})
    
    Data['% US Native'] = Data['US Native'] / Data['Total Population'] * 100
    Data['% Foreign Born'] = Data['Foreign Born'] / Data['Total Population'] * 100
    
    AllData.append(Data)

Datacombined = pd.concat(AllData).sort_values(by='year').reset_index(drop=True)

row2013 = Datacombined.loc[Datacombined['year'] == 2013].iloc[0]
row2023 = Datacombined.loc[Datacombined['year'] == 2023].iloc[0]

Datasummary = pd.DataFrame([
    {
        'Description': 'Total Population',
        '2013': row2013['Total Population'],
        '2023': row2023['Total Population'],
        'Absolute Change (2013-2023)': row2023['Total Population'] - row2013['Total Population'],
        'Percent Change (2013-2023)': ((row2023['Total Population'] - row2013['Total Population']) / 
                           row2013['Total Population'] * 100)
    },
    {
        'Description': 'US Native',
        '2013': row2013['US Native'],
        '2023': row2023['US Native'],
        'Absolute Change (2013-2023)': row2023['US Native'] - row2013['US Native'],
        'Percent Change (2013-2023)': ((row2023['US Native'] - row2013['US Native']) / row2013['US Native'] * 100)
    },
    {
        'Description': 'Foreign Born',
        '2013': row2013['Foreign Born'],
        '2023': row2023['Foreign Born'],
        'Absolute Change (2013-2023)': row2023['Foreign Born'] - row2013['Foreign Born'],
        'Percent Change (2013-2023)': ((row2023['Foreign Born'] - row2013['Foreign Born']) / row2013['Foreign Born'] * 100)
    },
    {
        'Description': '% US Native',
        '2013': row2013['% US Native'],
        '2023': row2023['% US Native'],
        'Absolute Change (2013-2023)': row2023['% US Native'] - row2013['% US Native'],
        'Percent Change (2013-2023)': ((row2023['% US Native'] - row2013['% US Native']) / row2013['% US Native'] * 100)
    },
    {
        'Description': '% Foreign Born',
        '2013': row2013['% Foreign Born'],
        '2023': row2023['% Foreign Born'],
        'Absolute Change (2013-2023)': row2023['% Foreign Born'] - row2013['% Foreign Born'],
        'Percent Change (2013-2023)': ((row2023['% Foreign Born'] - row2013['% Foreign Born']) / row2013['% Foreign Born'] * 100)
    }
])

# Round all numeric columns to zero decimals and convert to int
for col in ['2013', '2023', 'Absolute Change (2013-2023)', 'Percent Change (2013-2023)']:
    Datasummary[col] = Datasummary[col].round(0).astype(int)

Datasummary.to_csv('Foreign Born 5-Year Estimate 2013-2023.csv', index = False)


#### Veterans